# LSTM

In [1]:
import numpy as np
import random, sys

 ##### 元となるテキストを読み込む
 学習時間が膨大になりそうだったので容量が1/20の夢十夜

In [2]:
path = "夢十夜.txt"
bindata = open(path, "rb").read()
text = bindata.decode("shift_jis")
print('コーパスの長さ:', len(text))

コーパスの長さ: 20207


In [4]:
# 一文字ずつバラバラにして文字にIDを振る
chars = sorted(list(set(text)))
print('使われている文字の数:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 文字→ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID→文字

使われている文字の数: 1135


テキストをmaxlen文字で区切って、その文の次に来る文字を記録する

In [6]:
maxlen = 20
step = 3
sentences = []
next_chars = []

In [7]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('学習する文の数:', len(sentences))

学習する文の数: 6729


In [8]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

#### テキストをIDベクトル化する
X：三次元（テキスト数、　1テキスト単位（今回は20）、 使用された文字の種類数）  
y：二次元（テキスト数、 使用された文字の種類数）

In [10]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [12]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## keras登場

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

Using TensorFlow backend.


モデルを構築する(LSTM)

In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

選択候補となる配列から値を取り出す関数

In [16]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

学習させて、テキストを生成する・・・を繰り返す

In [16]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('繰り返し=', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 訓練
    # ランダムにテキストのシードを選ぶ
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 多様性のパラメータごとに文を生成する
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('---多様性=', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('---シード="' + sentence + '"')
        sys.stdout.write(generated)
        # シードを元にテキストを自動で生成する
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 次に来る文字を予測
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 既存の文に予測した一文字を足す
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
繰り返し= 1
Epoch 1/1


/opt/conda/lib/python3.6/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


13666/13666 [==============================] - 20s - loss: 1.4365    

---多様性= 0.2
---シード="》えた。馬は前足の蹄《ひづめ》を堅い岩の"
》えた。馬は前足の蹄《ひづめ》を堅い岩のににいる。自分はどもものようなものを眺めて、真中に下へ行ったから、どうかと云うから、一人いろいで、天うになって、真白いできる。ほにどこへは自分は大きな声が高かかたで、た。そのから母は一人の盲に見せて、そうしていたののをへへ行った。
　　　　　　　第八夜

　このごると、たんの女を一生《ひ》き》のように思って、そうして、生きているように思って、そうしていた。
「こにある。
　この頃るを真白い男の、この大きな声が高から爺さんは、庄から先で行った。そうかんは、庄ちゃになった」と云った通ら水と云うような息《は》の中で、大変ながらいで、一分うと思いついろうかんだろ


　　　　　　　　　　　　　　　　　ころ》をてものの間に小がいると云うと思い男のかからていた。
　自分はそのからりものの間にない。自分は一つと思うと思った。
　自分はそのこに、大きな声が高いかった。自分は、船がらいて

---多様性= 0.5
---シード="》えた。馬は前足の蹄《ひづめ》を堅い岩の"
》えた。馬は前足の蹄《ひづめ》を堅い岩のににいる。そのから四はいろって、子供のようと思いる。口にきりものような息《から》っている。
　自分はそのまで、大きな声が高がらくなった。自分は一まと思って、間にある。
　やってもののを鳴ら出た。そのしきりものの根へ子である。片《か》き》には、金《から吹き出けてみを見てるともる。はどこかがられて、こうんで、
「ここんだ庄だと云う」と云いたんで、いるほにはたえない。たかんは人光いとに水《ひず》っていた。そのほに小のようになった。そのかにはあるときのを子《かったら》き》のあった分は、太いて綱に一きればら出しまった。
　自分はそのゃにに光っている通、背中では、手らいないで、一ぜさん思うなくうばかね」と思い出すか」、やから吹いた。
　自分はこの女はいつと思いなけ《ひと思》る。けれどももの中へ子をある。片《かしき》るで、生《あおいみ》を吹かた。自分はどこわらくってなっただろうに思いる。
　

---多様性= 1.0
---シー

　庄太郎は電女をして、そのまって、その弓うと、この先って、そのようにものをして、た。大ここごはどこつとりとうないで、っていた。
「ここ子でも我子をして、その森へ相ってしていた。
　して、この鋭子をして、ない。島がいる。この大きな手をしてらその、子供な無にはただから、どこりつって行った。そのから誰から覗《すん》げて、そうして、四から吹いただろだ。

　　　　　第八夜

　こん》ますますか」とかって、その下って、その弓ってもなかった。そのである。
「百年は手を細い仁子をあるたんだろうかと思わ》って、ころもものもないだろうに思いて、た。今にも戦を負いたい時、身は女はこの下が、
「ある鑿へ鑿っている。自分の姿がにぼ《となず》っていた。ちいと云うんだかと、なか

---多様性= 1.0
---シード="も》へ前へのめった。岩の下は深い淵《ふち"
も》へ前へのめった。岩の下は深い淵《ふち》の八りを大むのぞでた。自分は股《心く》のから腰《かぶ》を刹ると云っな。った。涙が前ろう来ておから、ながら爺て、

　真闇《な《こう》の画ると早く》の手は、暗らその杷なった。鞍《てお》れ、。黒い馬が頭を身けて鼻かって、ころうつと思って急って下そうから目ん》自分て頭であにならが」
　自分の大きな坐を鳴い時時、もののなったに殺していてい子を自分は、不も立っていないる。騒にてるとうんだねいもだか、方ない。自分はぶじょり水をてる。それで眼れと手たら｜してなくなった。
「どこ」つ手ますから死ない。
「負間して、このょっていた。鋏をして、この爺みんでゅっていた。
　乗ると開いたないるの人うに、片《かなく》がした。
　庄にはするかねんを、やの「こくある。大変な赤いでで、梟だと逢いた。しま、人ら、と二なをが、行った。
　女は、こうから誰を沈るぶ、》って待《ていた。自分はない足で、死異ない。

---多様性= 1.2
---シード="も》へ前へのめった。岩の下は深い淵《ふち"
も》へ前へのめった。岩の下は深い淵《ふち》の容いた。
　その頃ように就いとのいるで殿って「なか」」とに十曇がじろ、
　しょれる子のから足で

真負《
》や突凄《はみ》のひしけ》に水を真こう出ちで、水拭を子をしてのせて、一通ておといた。自分は、太でといつついも方だね」ん
　-ようなごしてみ子のように、ょうようにるえす。その片は黒った頭い大を細

《け》っている。馬は蹄《ひづめ》の音が宙である。白に聞ろう云う自分は、で、ようち子屋へ口った。身は、深かな男が、こうにている。
「十間、このご　　第八直《こっどり下》のように見える。

「ある。
「「父」とくよ」。そうして、この根ようない七た。勘いてい人に御そう》に思ってもない。った。身はと隣や相がちちが、見てしあった。
　いるのの鞘が国って」って、》を突応にら、ながら待ってかった。
　自分はなだら、突んな座した。その時母が口るないろうにである。
　自分は大きな逢も見本の方た時でも電わなとったと思い出しでも水をずしている。自分し判には手らない時だ時、と云っ声が、相た「風く見てる方めみが直いた。
　いて、肥石の下へあるしいりと思っている。ただ一げんだんだかと云った。その危てういと、「ふうとんき》のように立って急ると、いちがいった。人は、背中で、長い時が面にこわら、ようにある。
　女は異生を沈け》いた。そうして、生

---多様性= 1.2
---シード="《け》っている。馬は蹄《ひづめ》の音が宙"
いけ》っている。馬は蹄《ひづめ》の音が宙であっパ風う間している。弓ったらい、女のったへしきななけれた。ううとらくないない黒ら自分はそうしている。るかと匂《く削やし》の括《は》》を目めて、かって行った。そのほとはいた。：ころぶら、」とた

　
　「唇って来た。そのほ《のみとみな仁めただね、身の上やあた。
　その方ようを見て、水の感をかった。覗れと見た自
　

　］定《めぶりった日その高売　らいた。弓もの顔を見た。しまい一らと水を見る。しまい逢はし》やせて、言っとくるようと思い態時、、ももななが」と小いあがいた。早ら、は、ここ赤ら死ぬないつなかった。その動かに、たかかんで、いろのを親て十中である。
　に人をそのほるをった。
　女は、生《ていたち》のそうにてるったなった。自分を見ていもの直《あった棒》の中で、こうように、持って行ぐよかなかってしまり開いた。自分は身がい分に帳いて百ずを郎している。自分は悟っていて待っ

--------------------------------------------------
繰り返し= 14
Epoch 1/1
13666/13666 [==============================] - 20s - loss: 0.07

...

「さすがは運慶だ方に、よういんだねて、いう上に乗って行た。その頃とはらと思った。
「さああはこの時始めて行った。そのほる、は、とはいを自分のごに思いるようい思いる。
　自分はまますわら、
　　　　の第十夜

　こんらこう女はいった。
「こん」ところ》ものをあらない。
　立っていためが、生きていじいない。いか大だながが、いろい。自分は手また一つと女を真黒い眉《こおじゃし》のでうか立ってのます足でき大だな夜いろう。い入いるとはただと、上からくなけない。た。けれどもももさいはは不ないろいる。天はから自分はあった。その「ある。ねへまでも、あやかただに、い。

　　　　　第八夜

　　　　　第八夜
　　　　　女

　　　　　　　　


　口闇《たまままた。そうから誰のらりと帰りいた。それでは、とういと水い。た
「あっありがら、女はたっしば》をして、生《てい》している。広の方

---多様性= 1.0
---シード="自分の方を振り向いて、
「さすがは運慶"
自分の方を振り向いて、
「さすがは運慶だかられで、生ち小いと船い小た。大い《はが》の手い時《はさい》の深かけて木い背。
「んある」
　、爺に》はたから持をじへと落たと、ち《はりち》の流れた。
　し上はを弓　庄て、こう「いって、水のほった手たと思うなる。
　爺さんは庄太「の水とあるた。
「ここ眼はだっとうころい。
「あるはらわな男から、弓すように思った。それで運慶だ々にきになった。自分は股れた。赤いんは死んなかなかった。
　いるもにものかね」とかった。
「ほのある方へいる。そのでも恋の腰から》を自分は大きなくつかとう上が、
「ある夕ずきない。かっている。
　自分はそれ水そうして、この船に息って自分はまた生いついはだかない。ただ背で来た茶また通で、たちま男は、｜ら》にも判だから、腰うからうな、女分の一面白底じうない。　の中に埋《くままますからそうから、生《うい》していた。で、一度《夜こりごは》のでの大へない。

---多様性= 1.2
---シード="自分の方を振り向いて、
「さすがは運慶"
自分の方を振り向いて、
「さすがは運慶だ方に、るようと云って「そう爺へんへ腰もりじきりかた。けれどもしから女を真たしまで来る。

　ころところううに、若いつま入のじいない。そのでは巻のお》であるた。しまいその白にらである。
　ひととしる。

》いた。女は静かな調子を一段張り上げて、
　百ほなへがらい。
　　　　　第十夜

　こんんきな手中になった。
　自分はままた我ら思いじいたんだろい。

　子供なるような暗子を、ねんで
　子で、運またらくる目れと云った。その時母が口《こん》くつ込《うろうとそうして、もの上になって、中ら自をす上へ行くと聞いた。爺さんは長庄ふの破をりとの木し命しち恐ろう。て　しまから庭らだら、で自分はた風や　自分はままたらその英に、たうち運さっと云った。聞だとはたなから、たんどうとていたの


　男を白いと面い》の間に大きい膝《ひお》く足た。》はあった。その時身はいろいた。自分の足が、思うと云う自せ上にもの
「さっとはだ。と思い男が、
「さんはだらとない。そうして、分のごである。
　自分は一れたかりいし女を時ってみ《前み》していた。共に目懸うに《いたやに足いし》がでまるでひ死ぬの前へあるの云った。
「ここんはだっ」としま

---多様性= 1.2
---シード="》いた。女は静かな調子を一段張り上げて、"
》いた。女は静かな調子を一段張り上げて、
　百」

　　　郎は、船らじいこの、運慶うとは電ややった。
　あうそのなった。いる。天らな女を自分の足である。その女は田の大ちい大いた。
　自分はそのをむい来て、じいと「さっんが」
　子供かまったらいる。びにている。すると、製が上で、やちで自分い頭もせにもまし女をいていた。。だだ。す精どこここんは》をてのやかとりととくようが、ゅり《かす四ようばなった。
　自分はままたきき思き残むと方り御引《あっっ》へ背いている方へいた。大変なくが、でよほど拝はにもりいつも拝い。たら容で、へっしだだと思いないである。ただ大はな長に遠《い身は女えかいて9の上が下る。ただ背は、長《ふら》な息を白いる。は、こりろし上が云って、また切って鞍きの来と船な事で、左って見てるとじまい風た。はせと眼のでき出きなない。双は時この犇ってなった。その手《またじ》くなくなる。のたますは、こうを―に来《かた並》を身う

--------------------------------------------------
繰り返し= 35
Epoch 1/1
13666/13666 [==============================] - 21s - loss: 0.0273 

　こんら柑ものこ　真》を仁って、ならいと独をた。して鋏人て、焼って弓まで、急にときるで、行ってかっ」とある。ただ行って、船の運慶
　立将をよくかけて年、あの上に下るような穿持て自」と相

--------------------------------------------------
繰り返し= 43
Epoch 1/1
13666/13666 [==============================] - 21s - loss: 0.0179    

---多様性= 0.2
---シード="てる見物人とはまるで釣り合が取れないよう"
てる見物人とはまるで釣り合が取れないようである。自分はどこからこったんなったばからやって見ると、持って、たちと見きる。
「どこもある。
「どこもまだすすと、一つて見ていた。
「ここらその爺には手のを上である。
　自分はたまた思い小中に入る出いた。
　しまも子供は、や《ちち》ってのやにながのった。自分はたまた一人いと云った。その「ある。
「ここ」て爺さんは、もの土を落ちておいと、。

　男爺さんは
「「今にあっすぐすか」と、どこの手をする。それで、急にていた。
「どこもある。
「どこもまだ」とわから、


爺さんは
「「今に」ら、ものは、船にない。
　その手を寸るで、一度と見えながら、どころう忽がの手を見て、眼を開けてた。その時なが子、口をしている。
「どこもまだ子をして、ない。ただ背で、をじていた。
「どこもある。
「どこもある。
「どこもある。
「どこもある。
「どこもある。
「ど

---多様性= 0.5
---シード="てる見物人とはまるで釣り合が取れないよう"
てる見物人とはまるで釣り合が取れないようである。自分はどこから自分は思って、なった。
　しの男をからて「」と眼を真白い男は自分はままそのでき込込んで、、もう少し女が、い《うだ方ている。その時《はと》で、片《かおじ》で、船の出りやって行った。
　して、もののように闇いていたんだなんの人がのであった。
　自分はたまた思い思うを見ている。そのから、たの上があった。
　とのようで、ものはただいて、ない。島中に思くずきない。
　して、女を帯った。
　柳のを手って見ていた。その口に、一の上に盲目て行くようと云った。庄ところが忽が供のほたと云った。そのほるものかと思って、

　こんは夢をた。その悟い一は、とろい。自分はそれたもののやになるは、っっない。
だろう。その悟って、、仰う《あちゃき》が、大きな鈴の輪がら自分は大きなら、と分《太郎は大ななのががらそうして、ものはなっ」と云った。ければもものうかだろいも、天さいまを一度《の》して、

　爺さんは

　爺さんは、


をそのとこのらる。すると、電の上に立って、唱

　運慶またんにきき》のょった。窓かなく黒を時なかって、一はあ《ん御うな》の女はままたら、あの切に一るもその見ている。片はと見える。たかと云う時、もってこねんなと帰
　、女はまったったまま足を大将ななけ」」とかな夢をた。とは、このでは、夫---------------------------------------------------------------------------------------------------

---多様性= 1.0
---シード="った。

　　　　　第四夜

　"
った。

　　　　　第四夜

　こんは9《た土った土
　
　　　　　第十夜

　こんり夢であった。
　自分は大きな声でもだからい。する。とうしてもいほと
「さっははだで、でもるとさった。母の手を子《こりせていた。
　庄すにはの人をものだからい。
　すると白うもの間に小えがののうで、左していたがに、うびある。
「日を箱のままでの上に乗《らい》に突わ、、着ていた。
「ここへはると
「見て、急になって、どうして、生《になきが》をして、金魚やは自分は大将な太夜が」との《む》っていた。しきを見る、ものかに旦って、ど》して自分ついて、

　爺さんは、庄うしていたが、女になっすがらいて鶏《鳴いと思がぶくて、遠くしている。眼をてく黒いして、その口にになかけて細た。
　さんは
「
「「さん」
　残999。自分は腰かたりかけて行った。その方へびっていないも命の間に二方へ行っくなんしか。天うして、生きにきときな

---多様性= 1.2
---シード="った。

　　　　　第四夜

　"
った。

　　　　　第四夜

　こんら夢をたをといる。自分は
たんだね」
　子供はた寸》った土だからばらろうか》っている。ら、ともな夜
　します子供に足《さ》》がある。
　立って将った。

　　　　第十夜
　こんら夢をたっざもは」と小をしての女に唄《つい》ま

　いかな庄太郎でも、あんまり呑気《のんきいずきの頭を見いて、
「さんは年は
「なはつをを健さん船、女はままも通たにゅうな船に森へ向いた

　　　　庄つる、船の

　　　　第八夜

　こんは親るの行く----------
あ　ぎうら、いる、無うにといている。そので小夢を見た。この女には、ぎいて、こうとらきなかった。
「いつぜつとあると、生らてい仁文がのじま
どやっている。無うちて行がらそのと云ったの、するとうち思うわちぐと、ないは行ってれをして、----------------------》るどいた。のいつみって削のを間から素で、ょっと

底爺さんる
「鳴らさってのを見て、四隣----------------------------------------------------



あ　　　　往大ない。

　　　　　第八夜

　こんな夢を》った。鋏《ひす》にひらないに


多様性確認

In [46]:
print()
print('-' * 50)
model.fit(X, y, batch_size=128, epochs=1) # 訓練
# ランダムにテキストのシードを選ぶ
start_index = random.randint(0, len(text) - maxlen - 1)
# 多様性のパラメータごとに文を生成する
for diversity in [0.01, 1.0, 100]:
    print()
    print('---多様性=', diversity)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('---シード="' + sentence + '"')
    sys.stdout.write(generated)
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    # シードを元にテキストを自動で生成する
    for i in range(200):
        # 次に来る文字を予測
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        # 既存の文に予測した一文字を足す
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

# 選択候補となる配列から値を取り出す
# def sample(preds, temperature=1.0):
def sample(preds, temperature):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


--------------------------------------------------
Epoch 1/1
13458/13458 [==============================] - 20s - loss: 0.3170    

---多様性= 0.01
---シード="健さんは庄太郎のパナマの帽子が貰いたいと"
健さんは庄太郎のパナマの帽子が貰いたいと、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、、

---多様性= 1.0
---シード="健さんは庄太郎のパナマの帽子が貰いたいと"
健さんは庄太郎のパナマの帽子が貰いたいと、、、、、も、、、、、、、、、、思、、、もも、、、、、柱、、、は、、、》、、、、、、、もも、、、、、、、、も小、、、、、、、、もも、、、、来、、も女も行、、、思、》、、、、、、、、、、、、、、、、云も、、、、も思、、、、、も、、、だ、云、云、、、、、、、、、、、、、思、、云、、、》、》、、、、、、、、、、も、も、、》、、、、、、、、、、、思い、、、、、六、思、、、、、、、、思も、、、も、、もも、、

---多様性= 100
---シード="健さんは庄太郎のパナマの帽子が貰いたいと"
健さんは庄太郎のパナマの帽子が貰いたいと降白考削るハ影牛帰仁賽前囃熊折目6負階突頓木唐覗苔選悟破探庫腹蜜茫倉空準歩9縮路杷だ鬣河攫粧縛仕若へ理特戈将。休食奴楫
握側つど宅挾散護キ西加檳脱ー粧場丸図町応鶏蔵意烈自耳尽い令矢梟拝叭ボ布早紐只えげ進げ浸先擲素納梶様真鋭膨三豆取っ敵垂腰」腐半定帳感揉後針ッ袍直梟焼屈静弾乗パ納精砕伏夢払暴が吹事粟こを窓枕汲全膳ピ束鈴も山運っ叭書鍮巨続男時ずt花吐り郭座ア極倚菓宅習添ま徹丈鮫髭九爭拍琥水旦庫墓頃正花
